In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [3]:
from catboost import CatBoostClassifier

import pickle

with open(data_path + 'model/step1_SMOTE.dat', 'rb') as fp:
    step_1 = pickle.load(fp)
with open(data_path + 'model/step2_OPTUNA.dat', 'rb') as fp:
    step_2 = pickle.load(fp)
with open(data_path + 'model/step3_Op_SMOTE.dat', 'rb') as fp:
    step_3 = pickle.load(fp)
    scaler = pickle.load(fp)
with open(data_path + 'model/step4_model.dat', 'rb') as fp:
    step_4 = pickle.load(fp)

In [4]:
x = pd.read_parquet(data_path + 'processed/test_12.parquet')
result = pd.read_csv(data_path + 'sample_submission.csv')

In [5]:
x.drop(['ID', '기준년월'], axis=1, inplace=True)

In [6]:
x_scaled = scaler.transform(x)

In [7]:
proba_1 = step_1.predict_proba(x_scaled)[:,1]

In [8]:
e_threshold = 0.642
d_threshold = 0.320
ab_threshold = 0.5 #0.723
b_threshold = 0.271
y = pd.DataFrame()

In [9]:
y['E'] = (proba_1 >= e_threshold).astype(int)

In [10]:
mask = y['E'] == 0

In [11]:
x_2 = x_scaled[mask]
proba_2 = step_2.predict_proba(x_2)

In [12]:
col1 = pd.Series(0, index=y.index)
col2 = pd.Series(0, index=y.index)
col1[mask] = (proba_2[:,1] >= ab_threshold).astype(int)
col2[mask] = 1 - (proba_2[:,1] >= ab_threshold).astype(int)

In [13]:
mask &= col2 == 1
x_3 = x_scaled[mask]
proba_3 = step_3.predict_proba(x_3)

In [14]:
colC = pd.Series(0, index=y.index)
colD = pd.Series(0, index=y.index)
colD[mask] = (proba_3[:,1] >= d_threshold).astype(int)
colC[mask] = 1 - (proba_3[:,1] >= d_threshold).astype(int)

In [15]:
mask = (y['E'] == 0) & (col1 == 1)
x_4 = x_scaled[mask]
proba_4 = step_4.predict_proba(x_4)

In [16]:
colB = pd.Series(0, index=y.index)
colA = pd.Series(0, index=y.index)
colB[mask] = (proba_4[:,1] >= b_threshold).astype(int)
colA[mask] = 1 - (proba_4[:,1] >= b_threshold).astype(int)

In [17]:
y['A'] = colA
y['B'] = colB
y['C'] = colC
y['D'] = colD

In [18]:
y.value_counts()

E  A  B  C  D
1  0  0  0  0    86917
0  0  0  0  1    10903
         1  0     1956
   1  0  0  0      217
   0  1  0  0        7
Name: count, dtype: int64

In [19]:
result['Segment'] = y.idxmax(axis=1)
result.to_csv(data_path + 'submission3.csv', index=False)